In [1]:
import pandas as pd
from scipy.stats import pearsonr
import os




In [2]:
#param
allinonefile="data/diffDataExpanded_SigMatrixpy_g50Onlyposition_intersected_cyberin_scaled_inverted_selectedinverted_CSxOut.txt_rendata_gt3.txt_oneline"
gtpath="dummyfolder/gtfolder"
allgtlist=os.listdir( gtpath )#["blueprint_rein_our_pbmc_puregt.txt","rendata_gt3.txt"]
gtfilename="rendata_gt3.txt"
purgtfilename="blueprint_rein_our_pbmc_puregt.txt"
masterfilename="data/Masterperf.txt"
allinonedf=pd.read_csv(allinonefile,sep="\t")
print(allgtlist)

['.DS_Store', 'rendata_gt3.txt', 'blueprint_rein_our_pbmc_puregt.txt', 'faltu.txt']


In [3]:
def metric_for_mixture(df):
    corr = pearsonr(df['ground_truth'], df['predicted'])
    return corr[0]


In [4]:
def metric_for_pure(df):
    diff=df['ground_truth']-df['predicted']
    
    
    result=1-(diff.abs()).mean()
    return result

In [5]:
def savetomastermat(df,gtfilename,purgtfilename,masterperf,dffilename):
    
    
    if gtfilename==purgtfilename:
        metric=metric_for_pure(df)
    else:
        metric=metric_for_mixture(df)
        
        
    smname=os.path.basename(dffilename)
    
    smname,sep, tail=smname.partition('_CSxOut')
    
    if os.path.exists(masterperf):
        masterperfdf=pd.read_csv(masterperf,sep="\t")
        
        
        if smname in masterperfdf["Sm"].tolist():
            
            
            masterperfdf.loc[masterperfdf["Sm"]==smname,gtfilename]=metric

        
        else:
            
            dic={}
            dic["Sm"]=smname

            for cl in allgtlist:
                dic[cl]="NA"


            masterperfdf=masterperfdf.append(dic,ignore_index=True)

            masterperfdf.loc[masterperfdf["Sm"]==smname,gtfilename]=metric
        
        
    else:
        masterperfdf=pd.DataFrame(columns=["Sm"]+allgtlist)
        dic={}
        dic["Sm"]=smname
        
        for cl in allgtlist:
            dic[cl]="NA"
            
        
        masterperfdf=masterperfdf.append(dic,ignore_index=True)
        
       
        masterperfdf.loc[masterperfdf["Sm"]==smname,gtfilename]=metric
        

        
        
        
      
    masterperfdf.to_csv(masterperf,sep="\t",index=False)

In [6]:
savetomastermat(allinonedf,gtfilename,purgtfilename,masterfilename,allinonefile)